<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/13_WDPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract Protected Areas from World Database

## Import gee and authenticate

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Input arguments for data extraction

In [ ]:
# Area of interest
aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_NS_buff_5km")
aoi_name = "MKR_NS_buff_5km"
layer_name = "RS_040"

PAs = ee.FeatureCollection("WCMC/WDPA/current/polygons").filterBounds(aoi)
output_name = f"{layer_name}_{aoi_name}"


## Process and extract PAs

In [ ]:
# Data set contains linestrings and polygons. Export can only handle one geometry
# Assign geometry and then filter only polygons

def assign_geoms(fc):
  return fc.set('geom_type', fc.geometry().type())

PAs = PAs.map(assign_geoms)

PA_polygons = PAs.filter(ee.Filter.equals("geom_type", "Polygon"))


In [ ]:
print(PAs.geometry().type().getInfo())
print(PA_polygons.geometry().type().getInfo())

In [ ]:
task = ee.batch.Export.table.toDrive(
    collection = PA_polygons,
    description = "EXPORT IMAGE TO DRIVE",
    fileFormat = "SHP",
    fileNamePrefix = output_name,
    folder = "GEE_exports"
    )

task.start()

In [ ]:
task.status()